In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import dash_table
import dash
import pandas as pd
import json
import csv


In [2]:
# Build App
app = JupyterDash(__name__,suppress_callback_exceptions=True)

#function to query dbpedia endpoint
def sql(value, endpoint):
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(value)
    sparql.setReturnFormat(JSON)
    processed_results = sparql.query().convert()
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

#layout

app.layout = html.Div([
    
   html.Div(
        className="navbar",
        children=[ 
            html.H3(
                children="Interactive Data Analytics Dashboard for RDF Knowledge Graphs", 
                className="navbar--title"),
            html.Div(
                children=[
                dcc.Tabs(id='tabs', value='Dashboard', children=[
                    dcc.Tab(label='Dashboard', value='Dashboard'),
                    dcc.Tab(label='Tables', value='Tables'),
                    dcc.Tab(label='Charts', value='Charts'),
                    dcc.Tab(label='Query Samples', value='Samples'),
                    dcc.Tab(label='Tutorial', value='Tutorial'),
                ]),
                html.Div(id='tabs-content', className="tabs-content")
            ])
    ]),
    html.Div(
        className="querybox",
        children=[
            html.H4(
                className="querybox--title",
                children="SPARQL Query"
            ),
            dcc.Textarea(
                id= "query-endpoint", 
                value="https://query.wikidata.org/sparql",
                placeholder="Enter your SPARQL query endpoint.",
                className="querybox--endpoint"
            ),
            dcc.Textarea(
                id= "query-text", 
                value="",
                placeholder="Enter your SPARQL query.",
                className="querybox--textarea"
            ),
            html.Button(
                id="submit-btn", className="querybox--btn", 
                children="SUBMIT", 
                n_clicks=0
            ),
            html.Button(
                id="compare-btn", className="querybox--btn", 
                children="COMPARE", 
                n_clicks=0
            ),
            html.Button(
                id="upload-btn", className="querybox--btn", 
                children="UPLOAD", 
                n_clicks=0
            ),
    ]),      
    #hidden div 
    html.Div( 
    id='intermediate-value', 
    children=[],
    style={'display': 'none'})
            
])#layout div


#-----------------------------------------query samples Callback----------------------------------------------------------------
#make query examples table from csv file
with open('../data/Book3.csv', 'rt') as f:
    csv_reader = csv.reader(f)
    headers = []
    queries = []
    endpoints = []
    for line in csv_reader:
        headers.append(line[0])
        queries.append(line[1])
        endpoints.append(line[2])

    data = {'Query Examples':  headers}
    df = pd.DataFrame (data, columns = ['Query Examples'])
    ResultListdataframe = pd.DataFrame (data=[], columns = [])


#-----------------------------------------initial empty table ----------------------------------------------------------------
#initial empty table
start_table_df = pd.DataFrame(columns=[''])

#-----------------------------------------selected tab callback ----------------------------------------------------------------

#render page for selected tab
@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value'),
)

#---------------------------------------------------render content--------------------------------------------------------------

def render_content(tab):
    if tab == 'Dashboard':
        return html.Div([
        ])
    elif tab == 'Tables':
        return html.Div([
            html.H3('Tables'),
            html.Div(
                id='MainTableDiv',
                children=[
                    dash_table.DataTable(
                        id='Resulttable',
                        data=start_table_df.to_dict('records'), 
                        columns = [{'id': c, 'name': c} for c in start_table_df.columns],
                        style_cell=dict(textAlign='left'),   
                        editable=True,
                        filter_action="native",
                        sort_action="native",
                        sort_mode="multi",
                        column_selectable="single",
                        row_selectable="multi",
                        row_deletable=True,
                        selected_columns=[],
                        selected_rows=[],
                        page_action="native",
                        page_current= 0,
                        page_size= 10,
                        export_format="csv"
                    )
                ],
                style= {'display': 'none'}
            ),
        ])
    elif tab == 'Charts':
        return html.Div([
               html.H3('Charts'), 
               html.Div(
                   id="result-graph",
                   className="result-graph",
                   children=[
                        #dcc.Dropdown(
                        #id='dropdown1',
                        #options=[{'label': col, 'value': col} for col in  ResultListdataframe.columns],
                        #multi=False,
                        #style={'width': '40%'}
                               # ),
                        #dcc.Dropdown(
                         #id='dropdown2',
                         #options=[{'label': col, 'value': col} for col in  ResultListdataframe.columns],
                         #multi=False,
                         #style={'width': '40%'}
                         #),
                       
                        dcc.Dropdown(
                        id='chooseChart-dropdown',
                        options=[
                        {'label': 'scatter chart', 'value': 'scatter'},
                        {'label': 'bar chart', 'value': 'bar'},
                        {'label': 'line chart', 'value': 'line'},
                        #{'label': 'pie chart', 'value': 'pie'},
                        #{'label': 'polar chart', 'value': 'polar'}


                        ],
                        value = 'bar',
                        multi=False,
                        style={'width': '40%'}
                        ),
                       
                        dcc.Graph(
                         id='graph',
                        ),
                
                     ],
                 style= {'display': 'none'}  
               )
  
        ])
    elif tab == 'Samples':
        return html.Div([
            html.H3('Query Samples'),
            html.Div(
                className="examples",
                children=[
                    dash_table.DataTable(
                        id='querySamples',
                        columns=[{"name": i, "id": i} for i in df.columns],
                        data=df.to_dict('records'),
                        style_table={'width': '800px'},
                        style_cell=dict(textAlign='left'), 
                        column_selectable='single'
                    )
                ])
        ])
    elif tab == 'Tutorial':
        return html.Div([
            html.H3('Tutorial'),
            html.Div(
                className="tutorial",
                children=[
                ]),
        ])

#---------------------------------------------------Query Samples Callback--------------------------------------------------------------
# write selected query sample in the query text area    
@app.callback(
    [Output('query-text', 'value'), Output('query-endpoint', 'value')],
    Input("querySamples", "active_cell")
)
def get_active_cell(active_cell):
    return queries[next(iter(active_cell.values()))], endpoints[next(iter(active_cell.values()))] 

#---------------------------------------------------Get-data Callback--------------------------------------------------------------
@app.callback(Output("intermediate-value", 'children'),
              [Input("submit-btn", "n_clicks")],
              [State("query-text","value"), State("query-endpoint","value")])

def get_data(n_clicks, value, endpoint):
     if n_clicks > 0:
        ResultListdataframe = sql(value, endpoint)   
        
        return ResultListdataframe.to_json(date_format='iso', orient='split')


#---------------------------------------------------Table Callback--------------------------------------------------------------
#generate table from query    
@app.callback(
    [Output("Resulttable", "data"), Output('Resulttable', 'columns'), Output('MainTableDiv', 'style')],
    [Input("submit-btn", "n_clicks"),
    Input("intermediate-value", "children"),],
)    
def gen_table(n_clicks, jsonified_ResultListdataframe):

    if n_clicks > 0:

        resultListdataframe = pd.read_json(jsonified_ResultListdataframe, orient='split')
        mycolumns = [{'name': index, 'id': index,"deletable": True, "selectable": True} for index in resultListdataframe.columns]

        return resultListdataframe.to_dict('records'),mycolumns,{'display': 'block'}
    return start_table_df.to_dict('records'), [{'id': '', 'name': ''}],{'display': 'none'}

#---------------------------------------------------Charts Callback--------------------------------------------------------------
@app.callback(
    [Output("graph", "figure"), Output('result-graph', 'style')],
    [Input("submit-btn", "n_clicks"),
    Input("intermediate-value", "children"),
    Input("chooseChart-dropdown","value")],
)
def gen_graph(n_clicks, jsonified_ResultListdataframe, value):
    fig = html.Div()
    if n_clicks > 0:
        #print(value)
        ResultListdataframe = pd.read_json(jsonified_ResultListdataframe, orient='split')
        #for val in value:
        if(value=='scatter'):
            fig=px.scatter(ResultListdataframe,x=ResultListdataframe.columns[0], y=ResultListdataframe.columns[1])
        elif(value=='bar'):
            fig=px.bar(ResultListdataframe,x=ResultListdataframe.columns[0], y=ResultListdataframe.columns[1])
        elif(value=='line'):
            fig=px.line(ResultListdataframe,x=ResultListdataframe.columns[0], y=ResultListdataframe.columns[1])
        #elif(value=='pie'):
        #    fig=px.pie(ResultListdataframe,value=ResultListdataframe.columns[0], names=ResultListdataframe.columns[1]) 
        #elif(value=='polar'):
        #    fig=px.polar(ResultListdataframe,r=ResultListdataframe.columns[0], theta=ResultListdataframe.columns[1])
        return fig,{'display': 'block'}
 

In [3]:
# Run app and display result inline in the notebook
app.run_server(mode='external',port=8090)

Dash app running on http://127.0.0.1:8090/
    yearmonth count
0      2016-7  1892
1      2006-6  1396
2     2015-12  1881
3     2007-12  1612
4      2014-2  1821
..        ...   ...
248    2015-8  1803
249    2017-9  1732
250    2018-7  1643
251    2018-2  1828
252    2016-6  1679

[253 rows x 2 columns]
